### ライブラリ、データのダウンロード、データの抽出

In [1]:
import numpy as np

# 前処理にはsklearnライブラリを使っていきます

from sklearn import preprocessing
# データを読み込みます
raw_csv_data =np.loadtxt('Audiobooks-data.csv',delimiter=',')

# 最初のIDの列と、ターゲットである最後の列を取り除いた上で変数に代入します
unscaled_inputs_all =raw_csv_data[:,1:-1]

# 最後の列のデータをターゲットとして変数に代入していきます
targets_all = raw_csv_data[:,-1]

### targets_all のデータの型の確認

In [2]:
targets_all.shape

(14084,)

### 1と0の数を数えて、0と1の数が均等になるように、余分の0をリストに格納する

In [3]:
# ターゲットの数を数えます
num_one_targets =np.sum(targets_all) 

# 数を数えるための変数を定義します
zero_targets_counter = 0
# バランスを取る上で不要なデータを入れるためのリストを定義します
indices_to_remove = []

# ターゲットの数を数え、その数を超える0のデータがあった場合はリストにその値をいれていきます
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>=num_one_targets:
           indices_to_remove.append(i)

# 入力とターゲットを入れるための変数を定義します
# また、バランスが取れていないデータを削除していきます
#　0と1の数を揃えた上で全体のデータの数を揃える
unscaled_inputs_equal_priors =np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis=0)

In [4]:
num_one_targets

2237.0

### インプットデータを標準化する

In [5]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)

### データをシャッフルして、変数に格納する

In [6]:
# データをバッチに分けていくことから、何らかの規則性が生まれないようにデータをシャッフルしていきます
shuffled_indices =np.arange(scaled_inputs.shape[0])#行のみを取り出している
np.random.shuffle(shuffled_indices)
# シャッフルしたデータを変数に入れていきます(行をバラバラにしている)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets =targets_equal_priors[shuffled_indices]



### データの数を数え、訓練用、検証用、テスト用に分ける

In [7]:
# データの数を数えます
samples_count = shuffled_inputs.shape[0]

# 80-10-10に分けることを前提として、訓練用と検証用のデータの数を定義していきます(データ型も念の為変換した上で行う)
# Naturally, the numbers are integers.
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
# テスト用のデータの数を定義します
test_samples_count = int(samples_count-train_samples_count-train_samples_count)


# 訓練用データを作成します
train_inputs = shuffled_inputs[:train_samples_count]
train_targets =shuffled_targets[:train_samples_count]
# 検証用データを作成します
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets =shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]

# テスト用データを作成します
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_inputs[train_samples_count+validation_samples_count:]



# 作成したデータの中の１と０の割合を確認します
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)



1792.0 3578 0.5008384572386808
-69.06495890160508 447 -0.1545077380349107
12.270527420203045 -2683 -0.004573435490198675


### Ssavezメソッドを使ってデータを保存していきます


In [8]:
# Ssavezメソッドを使ってデータを保存していきます
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)


### 機械学習用に新たにライブラリーをインポートする

In [11]:
# 新たに一からコードを書いていきますので、再度ライブラリをインポートします
import numpy as np
import tensorflow as tf

# まずは訓練データを一時的に読み込みます
npz = np.load('Audiobooks_data_train.npz')

# 訓練用のデータを入力とターゲットに分けて変数に代入していきます
train_inputs =npz['inputs'].astype(np.float)#データ型も念の為変換した上で行う
train_targets =npz['targets'].astype(np.int) #データ型も念の為変換した上で行う



C:\Users\fujir\AppData\Local\Temp\ipykernel_10300\1408153987.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs =npz['inputs'].astype(np.float)#データ型も念の為変換した上で行う
C:\Users\fujir\AppData\Local\Temp\ipykernel_10300\1408153987.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: 

### 訓練データを読み込み、訓練用、検証用、テスト用でそれぞれ変数に代入する

In [12]:
# 検証用データを一時的に読み込みます
npz = np.load('Audiobooks_data_validation.npz')
# 訓練用のデータを入力とターゲットに分けて変数に代入していきます
validation_inputs, validation_targets =npz['inputs'].astype(np.float),npz['targets'].astype(np.int) #データ型も念の為変換した上で行う

# テストデータを一時的に読み込みます
npz = np.load('Audiobooks_data_test.npz')
# テスト用のデータを入力とターゲットに分けて変数に代入していきます
test_inputs, test_targets =npz['inputs'].astype(np.float),npz['targets'].astype(np.int) #データ型も念の為変換した上で行う

C:\Users\fujir\AppData\Local\Temp\ipykernel_10300\863552815.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  validation_inputs, validation_targets =npz['inputs'].astype(np.float),npz['targets'].astype(np.int) #データ型も念の為変換した上で行う
C:\Users\fujir\AppData\Local\Temp\ipykernel_10300\863552815.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depr

### TensofFlow で入力層、出力層等を定義しモデルを作成。フィットさせる

In [14]:
# 入力の数と出力の数を定義します
input_size = 10
output_size = 2
# 隠れ層のユニットの数を定義します
hidden_layer_size = 30
    
# モデルを作成していきます
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1番目の隠れ層
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2番目の隠れ層
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3番目の隠れ層
   
    tf.keras.layers.Dense(output_size, activation='softmax') # 出力層
])


# 損失関数と最適化アルゴリズムを定義していきます
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 訓練

# バッチサイズの設定
batch_size = 100

# 繰り返し回数の設定
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
# モデルにデータを入れていきます
model.fit(train_inputs, # 訓練データの入力
          train_targets, # 訓練データのターゲット
          batch_size=batch_size, # バッチサイズ
          epochs=max_epochs, # 繰り返し回数
            callbacks=[early_stopping], # アーリーストッピング
          validation_data=(validation_inputs, validation_targets), # 検証データの指定
          verbose = 2 # 結果の見た目を変更
          )  

Epoch 1/100


ValueError: in user code:

    C:\Users\fujir\anaconda3\envs\py-TF2.0\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 test_function  *
        return step_function(self, iterator)
    C:\Users\fujir\anaconda3\envs\py-TF2.0\lib\site-packages\tensorflow\python\keras\engine\training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\fujir\anaconda3\envs\py-TF2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\fujir\anaconda3\envs\py-TF2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\fujir\anaconda3\envs\py-TF2.0\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\fujir\anaconda3\envs\py-

### テストデータでテストを行う

In [ ]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))